# TOP相似的信息

## linux下获取内存信息

In [52]:
!free -m

             total       used       free     shared    buffers     cached
Mem:           993        918         75          1          5         49
-/+ buffers/cache:        864        129
Swap:         1023        626        397


直接限制的值并不是真正意义的值. 比如Mem这一行的`free`并不是说真正剩余内存. 下面说明一下上面值的含义:

#### Mem

- total 内存总数: 993
- used 已经使用的内存数: 918
- free 空闲的内存数: 75
- shared 当前已经废弃不用
- buffers Buffer Cache内存数: 5
- cached Page Cache内存数: 49

#### -/+ buffers/cache

- -buffers/cache 的内存数：864 (等于第1行的 used - buffers - cached)
- +buffers/cache 的内存数: 129 (等于第1行的 free + buffers + cached)

可见`-buffers/cache`反映的是**被程序实实在在吃掉的内存**，而`+buffers/cache`反映的是**可以挪用的内存**总数.


当你看见 buffer/cache 的空闲空间低或者 swap 的空闲空间低，说明内存需要升级了：
- 这意味这内存利用率很高.
- 代码太吃内存了,代码优化

In [19]:
# %load ../util.py
def bytes2human(n):
    # http://code.activestate.com/recipes/578019
    # >>> bytes2human(10000)
    # '9.8K'
    # >>> bytes2human(100001221)
    # '95.4M'
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for i, s in enumerate(symbols):
        prefix[s] = 1 << (i + 1) * 10
    for s in reversed(symbols):
        if n >= prefix[s]:
            value = float(n) / prefix[s]
            return '%.1f%s' % (value, s)
    return "%sB" % n


## 系统内存信息

#### 物理内存

> if you just want to know how much physical memory is left in a cross platform fashion simply rely on the available field.

`virtual_memory`读取的和`free`命令里值还是不一样. 所以这里使用`total`和`available`


In [136]:
import psutil
mem = psutil.virtual_memory()
print('mem:', mem)
print("物理内存:",bytes2human(mem.total))
print("可用内存(不需要交换):",bytes2human(mem.available))
print("内存使用率:",mem.percent)

mem: svmem(total=1042198528, available=162652160, percent=84.4, used=722202624, free=66297856, active=325947392, inactive=377892864, buffers=9388032, cached=244310016, shared=1740800)
物理内存: 993.9M
可用内存(不需要交换): 155.1M
内存使用率: 84.4


#### 交换分区

In [120]:
# swap usage
swap = psutil.swap_memory()
print('交换使用率:', swap.percent)
print('交换总量:', swap.total)
print('交换使用:', swap.used)


交换使用率: 70.2
交换总量: 1073737728
交换使用: 753819648


## CPU信息

In [109]:
print("CPU percent:", psutil.cpu_percent(interval=0))

CPU percent: 3.7


## 负载

负载是与cpu核心有关的. 

In [115]:
print('loadavg(1/5/15):', os.getloadavg())

loadavg(1/5/15): (0.05, 0.08, 0.12)


## 进程信息

进程信息字段说明:

- cmdline: mysqld
- cpu_percent: 0.5
- ctime: 32:18.53
- memory_percent: 16.1
- name: mysqld"
- nice: 0
- pid: 11231
- rss: 167632896
- username: vboxadd
- vms: 1155301376


相应的top输出结果:

```bash
  PID USER     PR  NI    VIRT    RES    SHR S %CPU %MEM     TIME+ COMMAND                                                                                                                              
11231 vboxadd   20   0    1128224 163704   3664 S  0.3 16.1  32:18.86 mysqld  
```

对应关系:
- rss-->res
- vms-->vms
- cmdline-->COMMAND
- cpu_percent--> %CPU
- ctime-->TIME+
- memory_percent-->%MEM
- nice-->ni
- pid-->PID
- username-->USER




In [119]:
import datetime
import os
import sys
import time
import psutil
import json


def get_process_infos():
    # 获取进程快照
    procs = []
    procs_status = {}
    for p in psutil.process_iter():
        try:
            p.dict = p.as_dict(['username', 'nice', 'memory_info',
                                'memory_percent', 'cpu_percent',
                                'cpu_times', 'name', 'cmdline','status'])
        except psutil.NoSuchProcess:
            pass
        else:
            procs.append(p)

    # return processes sorted by CPU percent usage
    procs = sorted(procs, key=lambda p: p.dict['memory_percent'],
                       reverse=True)

    
    process_infos = [
        
    ]
    for p in procs:
        # TIME+ column shows process CPU cumulative time and it
        # is expressed as: "mm:ss.ms"
        if p.dict['cpu_times'] is not None:
            ctime = datetime.timedelta(seconds=sum(p.dict['cpu_times']))
            ctime = "%s:%s.%s" % (ctime.seconds // 60 % 60,
                                  str((ctime.seconds % 60)).zfill(2),
                                  str(ctime.microseconds)[:2])
        else:
            ctime = ''
        if p.dict['memory_percent'] is not None:
            p.dict['memory_percent'] = round(p.dict['memory_percent'], 1)
        else:
            p.dict['memory_percent'] = ''
        if p.dict['cpu_percent'] is None:
            p.dict['cpu_percent'] = ''

        process_infos.append({
            'pid':p.pid,
             'username': p.dict.get('username', ''),
             'nice': p.dict['nice'],
             'vms': getattr(p.dict['memory_info'], 'vms', 0),
             'rss': getattr(p.dict['memory_info'], 'rss', 0),
             'cpu_percent': p.dict['cpu_percent'],
             'memory_percent': p.dict['memory_percent'],
             'ctime': ctime,
             'name':  p.dict['name'],
             'cmdline':  " ".join(p.dict['cmdline']),
     
            
        })
        
    return process_infos
process_infos = get_process_infos()
print(json.dumps(process_infos[0], sort_keys=True, indent=4))



{
    "cmdline": "mysqld",
    "cpu_percent": 0.5,
    "ctime": "32:18.53",
    "memory_percent": 16.1,
    "name": "mysqld",
    "nice": 0,
    "pid": 11231,
    "rss": 167632896,
    "username": "vboxadd",
    "vms": 1155301376
}


## 参考
- [free]( http://www.cnblogs.com/pengdonglin137/p/3315124.html)
- [psutil-top](https://github.com/giampaolo/psutil/blob/master/scripts/top.py)